#### Importação de bibliotecas

In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model, preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import warnings 
warnings.filterwarnings("ignore")

#### Importação das base de dados

In [169]:
dftrain = pd.read_csv('train.csv')
dftest = pd.read_csv('test.csv')

#### Verificação das correlações com a variável alvo (variável alvo: NU_NOTA_MT - Nota de matemática)

In [170]:
math_correlation = pd.DataFrame(dftrain.corr()['NU_NOTA_MT'].sort_values(ascending = False))

In [171]:
math_correlation

,NU_NOTA_MT
NU_NOTA_MT,1.000000
NU_NOTA_CN,0.584941
NU_NOTA_CH,0.529594
NU_NOTA_LC,0.494695
NU_NOTA_REDACAO,0.379376
...,...
IN_SALA_ACOMPANHANTE,NaN
IN_MOBILIARIO_ESPECIFICO,NaN
IN_MATERIAL_ESPECIFICO,NaN
TP_PRESENCA_LC,NaN


In [172]:
math_correlation = pd.DataFrame(dftrain.corr()['NU_NOTA_MT'].sort_values(ascending = True))

In [173]:
math_correlation

,NU_NOTA_MT
Q037,-0.265249
Q038,-0.202259
TP_LINGUA,-0.201726
TP_COR_RACA,-0.157364
Q036,-0.145501
...,...
IN_SALA_ACOMPANHANTE,NaN
IN_MOBILIARIO_ESPECIFICO,NaN
IN_MATERIAL_ESPECIFICO,NaN
TP_PRESENCA_LC,NaN


##### Por comodidade, serão utilizadas as quatro primeiras variáveis com correlação positiva mais alta:

In [174]:
x = dftrain[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']].fillna(-1000)
y = dftrain[['NU_NOTA_MT']]

#### Tratando os dados faltantes

In [175]:
x['NU_NOTA_LC'].fillna(-1000, inplace=True)
x['NU_NOTA_CH'].fillna(-1000, inplace=True)
x['NU_NOTA_CN'].fillna(-1000, inplace=True)
x['NU_NOTA_REDACAO'].fillna(-1000, inplace=True)
y['NU_NOTA_MT'].fillna(-1000, inplace=True)

#### Separando os grupos de treino e de teste

##### Foi utilizados um grupo de treino bem maior em relação ao de teste para se ter um modelo mais bem treino, afinal, este modelo só será utilizados na base de dados de teste (dftest)

In [176]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.99)

#### Aplicação do modelo de Random Forest

In [177]:
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

#### Atribuindo à variável Xres as variáveis da base de dados de teste

In [178]:
Xres = dftest[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']].fillna(-1000)

#### Fazendo o "fit" dos dados

In [179]:
random_forest = RandomForestRegressor(n_estimators = 100)
random_forest.fit(x, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

#### Predizendo os valores da nota de matemática com base na variável Xres que possui todas as variáveis usadas para predição

In [180]:
Yres = random_forest.predict(Xres)

##### A predição atribui valores negativos a certas notas com as variáveis Xres, isso se deve, por exemplo, às notas que são zeradas pelos alunos e o modelo prediz a nota de matemática como negativa. Para isso, foram atribuídas notas também zeradas para matemática, uma vez que não existe nota negativa para este caso

In [186]:
Yres[Yres < 0] = 0

#### Arredondando valores

In [182]:
dftest['NU_NOTA_MT']=np.around(Yres, 2)

#### Atribuindos as notas de matemática à base de dados de teste

In [183]:
dftest = dftest[['NU_INSCRICAO','NU_NOTA_MT']]

#### Transformando a base de dados em um arquivo .csv

In [184]:
dftest.to_csv('answer.csv', index=False)